In [19]:
import os
from typing import TypedDict, List, Optional
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import StateGraph, END
from constants import BASE_URL, API_KEY, MODEL_NAME

In [20]:
class AgentState(TypedDict):
    website_context: str
    channel_archive: str
    style_guide: str
    suggested_topics: List[str]
    selected_topic: str
    final_post: str
    skip_list: List[str]  # e.g., ["stylist", "writer"]

In [21]:
llm = ChatOpenAI(
    base_url=BASE_URL, 
    api_key=API_KEY, 
    model=MODEL_NAME,
    temperature=0.7
)

In [22]:
# Structured output schemas

class Topic(BaseModel):
    title: str = Field(description="Заголовок темы")
    tag: str = Field(description="Тематический тег, например #ai")
    preview: str = Field(description="Краткое описание того, о чем будет пост")

class TopicProposal(BaseModel):
    topics: List[Topic] = Field(description="3 уникальных темы для постов на основе профиля автора, и информации и контента предыдущих постов")

class DuplicateCheck(BaseModel):
    is_duplicate: bool = Field(description="True если тема уже была в канале, False если тема новая")
    reason: str = Field(description="Объяснение, почему тема считается дубликатом или почему она уникальна")

# Пишем самих агентов

def librarian_agent(state: AgentState, website_path='website_content.md', posts_path='posts.md'):
    """Reads local files to provide context."""
    with open(website_path, "r", encoding="utf-8") as f:
        web_data = f.read()
    with open(posts_path, "r", encoding="utf-8") as f:
        post_data = f.read()
    return {"website_context": web_data, "channel_archive": post_data}

def stylist_agent(state: AgentState):
    """Analyzes Russian writing style if not skipped."""
    if "stylist" in state["skip_list"]:
        return {"style_guide": "Нет запроса под конкрентный стиль написания поста"}
    
    prompt = f"Проанализируй следующие посты и опиши стиль изложения информации и написания постов:\n\n{state['channel_archive'][:2000]}"
    response = llm.invoke(prompt)
    return {"style_guide": response.content}

def planner_agent(state: AgentState):
    planner_llm = llm.with_structured_output(TopicProposal, method="json_mode")
    
    system_instructions = SystemMessage(content=(
        "Ты — эксперт по контенту. Отвечай ТОЛЬКО в формате JSON. "
        "Используй ТОЛЬКО ключ 'topics' для списка тем. "
        "Каждая тема должна содержать: title, tag, preview."
    ))
    
    user_prompt = HumanMessage(content=(
        f"Информация об авторе с его сайта (на английском): {state['website_context']}\n"
        f"Информация о предыдущих постах, и контент самих постов: {state['channel_archive']}\n"
        "Предложи 3 темы. Ответ строго по схеме: {'topics': [{'title': '...', 'tag': '...', 'preview': '...'}]}"
    ))

    result = planner_llm.invoke([system_instructions, user_prompt])
    return {"suggested_topics": result.topics}

def gatekeeper_agent(state: AgentState):
    check_llm = llm.with_structured_output(DuplicateCheck, method="json_mode")
    
    topic_obj = state["suggested_topics"][0]
    topic_to_check = f"Заголовок: {topic_obj.title}\nОписание: {topic_obj.preview}"
    
    system_instructions = SystemMessage(content=(
        "Ты — опытный шеф-редактор технологического блога. Твоя задача — следить, чтобы контент не был ВТОРИЧНЫМ, "
        "но при этом ты понимаешь, что важные темы (например, AI, обучение, карьера) могут и должны освещаться с разных сторон.\n\n"
        
        "КРИТЕРИИ ДУБЛИКАТА:\n"
        "1. Считай тему ДУБЛИКАТОМ (is_duplicate: true), только если новый пост предлагает ТОТ ЖЕ САМЫЙ совет, "
        "использует ТЕ ЖЕ примеры и делает ТОТ ЖЕ вывод, что и один из старых постов.\n"
        "2. Считай тему УНИКАЛЬНОЙ (is_duplicate: false), если она:\n"
        "   - Раскрывает подтему более глубоко.\n"
        "   - Предлагает новый практический кейс или личный опыт.\n"
        "   - Смотрит на ту же проблему под другим углом (например, раньше писали 'как начать', а теперь 'ошибки профи').\n"
        "   - Актуализирует старую тему новыми данными.\n\n"
        
        "Будь лояльным: если есть сомнения — пропускай пост. Отвечай ТОЛЬКО в формате JSON."
    ))
    
    user_prompt = HumanMessage(content=(
        f"АРХИВ КАНАЛА (предыдущие посты):\n{state['channel_archive']}\n\n"
        f"НОВАЯ ИДЕЯ:\n{topic_to_check}\n\n"
        "Задание: Сравни новую идею с архивом. Является ли она скучным самоповтором или это свежий взгляд на тему? "
        "Ответь в JSON: {'is_duplicate': bool, 'reason': 'краткий вердикт'}"
    ))
    
    result = check_llm.invoke([system_instructions, user_prompt])
    
    # Теперь логика возврата
    if result.is_duplicate:
        return {"selected_topic": f"ОТКЛОНЕНО (Дубликат): {result.reason}"}
    
    # Если тема прошла (или модель нашла нюансы), передаем её писателю
    # Добавляем комментарий редактора, чтобы писатель учел "свежий взгляд"
    enhanced_topic = f"{topic_to_check}\n\n(Комментарий редактора: {result.reason})"
    return {"selected_topic": enhanced_topic}

def writer_agent(state: AgentState):
    if "writer" in state["skip_list"]:
        return {"final_post": "Пропуск написания. Тема: " + state["selected_topic"]}
    
    # Если Gatekeeper отклонил тему
    if "ОТКЛОНЕНО" in state["selected_topic"]:
        return {"final_post": f"Пост не может быть написан. Причина: {state['selected_topic']}"}
    
    prompt = f"""
    Напиши пост для телеграм-канала, используя следующую информацию:
    Тема и превью: {state['selected_topic']}
    Стиль: {state['style_guide']}
    Контекст о создателе телеграм-канала с его сайта (на английском): {state['website_context']}
    Архив предыдущих постов телеграм-канала и их контент: {state['channel_archive']}
    """
    response = llm.invoke(prompt)
    return {"final_post": response.content}

In [23]:
# Строим сам граф
workflow = StateGraph(AgentState)

workflow.add_node("librarian", librarian_agent)
workflow.add_node("stylist", stylist_agent)
workflow.add_node("planner", planner_agent)
workflow.add_node("gatekeeper", gatekeeper_agent)
workflow.add_node("writer", writer_agent)

workflow.set_entry_point("librarian")
workflow.add_edge("librarian", "stylist")
workflow.add_edge("stylist", "planner")
workflow.add_edge("planner", "gatekeeper")
workflow.add_edge("gatekeeper", "writer")
workflow.add_edge("writer", END)

app = workflow.compile()

In [24]:
# Применяем пайплайн
initial_state = {
    "skip_list": [] # Add "stylist" or "writer" here to skip them
}

result = app.invoke(initial_state)
print("--- FINAL POST ---")
print(result["final_post"])

--- FINAL POST ---
**🌍 Сравнение культурных особенностей: Россия vs Китай**

Привет! Сегодня хочу поделиться наблюдениями о том, как жизнь, учеба и традиции отличаются в России и Китае — на основе своего опыта. Если вы когда-нибудь задумывались, насколько разными могут быть повседневные мелочи в двух таких больших странах, этот пост для вас.

### **🏠 Повседневная жизнь: кампус vs городская суета**
В России студенческая жизнь часто распределена по разным точкам города: одно здание на Покровке, другое — в двадцати минутах ходьбы, а общага вообще в Одинцово. В Китае (например, в Цинхуа) все сосредоточено в одном месте: учебные корпуса, общаги, столовки, спортивные площадки и даже музей с озером. Это создает ощущение закрытой экосистемы, где учеба и досуг переплетаются гораздо теснее.

*Вывод:* В России студенческая жизнь более "распылена", в Китае — компактна и интегрирована.

### **📚 Учеба: строгость vs самостоятельность**
В российских вузах (например, в ВШЭ) акцент на лекциях, семинарах